In [3]:
import geopandas as gpd


In [4]:
voivodeship_map = gpd.read_file(r'wojewodztwa\wojewodztwa.shp')

In [7]:
voivodeship_map = voivodeship_map.to_crs(4326)

In [8]:
voivodeship_map

,gml_id,JPT_SJR_KO,JPT_POWIER,JPT_KOD_JE,JPT_NAZWA_,JPT_ORGAN_,JPT_JOR_ID,WERSJA_OD,WERSJA_DO,WAZNY_OD,...,JPT_ID,JPT_POWI_1,JPT_KJ_I_3,JPT_GEOMET,JPT_GEOM_1,SHAPE_LENG,SHAPE_AREA,REGON,RODZAJ,geometry
0,None,WOJ,2289315,32,zachodniopomorskie,None,0,20220613,0,20120926,...,1364544,0,None,0,0,12.1323542862,3.10794349875,81168387600000,wojewodztwo,"POLYGON ((14.24969 53.92403, 14.27264 53.92417..."
1,None,WOJ,1398751,08,lubuskie,None,0,20211210,0,20120926,...,1361674,0,None,0,0,11.0914310011,1.83850176383,97789593100000,wojewodztwo,"POLYGON ((15.38223 51.51179, 15.38185 51.51153..."
2,None,WOJ,1994777,02,dolnośląskie,None,0,20221019,0,20120926,...,1365817,0,None,0,0,13.846094171,2.55916927233,93193464400000,wojewodztwo,"POLYGON ((16.91342 50.44976, 16.91210 50.44982..."
3,None,WOJ,2982700,30,wielkopolskie,None,0,20220330,0,20120926,...,1363546,2982700,None,0,0,18.4097920287,3.93220020515,63125781600000,wojewodztwo,"POLYGON ((18.14065 52.50609, 18.14115 52.50605..."
4,None,WOJ,1831001,22,pomorskie,None,0,20220906,0,20120926,...,1365552,0,None,0,0,11.7084107024,2.68945114136,19167483600000,wojewodztwo,"POLYGON ((18.82439 54.62561, 18.82619 54.62326..."
5,None,WOJ,941272,16,opolskie,None,0,20221205,0,20120926,...,1366338,0,None,0,0,9.00556550302,1.19620046778,53141242100000,wojewodztwo,"POLYGON ((17.74135 50.28711, 17.74138 50.28771..."
6,None,WOJ,1797058,04,kujawsko-pomorskie,None,0,20220330,0,20120926,...,1363545,0,None,0,0,12.3969283997,2.40941372674,09235061300000,wojewodztwo,"POLYGON ((18.14065 52.50609, 18.14011 52.50601..."
7,None,WOJ,1233406,24,śląskie,None,0,20220321,0,20120926,...,1363466,0,None,0,0,12.1292325536,1.55733578555,27625465000000,wojewodztwo,"MULTIPOLYGON (((19.83803 50.59001, 19.83754 50..."
8,None,WOJ,1821720,10,łódzkie,None,0,20211214,0,20120926,...,1361704,0,None,0,0,12.0791841816,2.36362021645,47205762600000,wojewodztwo,"POLYGON ((20.42486 51.61167, 20.42489 51.61162..."
9,None,WOJ,1518007,12,małopolskie,None,0,20221206,0,20120926,...,1366375,0,None,0,0,11.4286676945,1.8985108988,35155428700000,wojewodztwo,"POLYGON ((20.80878 49.35366, 20.80898 49.35375..."


In [19]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
from shapely.wkt import dumps


In [20]:
key_path = "permissions-417120-8c4dc3a5c773.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"],
)

client = bigquery.Client(
        credentials=credentials,
        project=credentials.project_id,
    )

table_id = 'permissions-417120.permissions_dataset.permissions_polygons_new_crs'

In [21]:
voivodeship_map['geometry'] = voivodeship_map['geometry'].apply(lambda geom: dumps(geom))

pandas_gbq.to_gbq(voivodeship_map, 'permissions-417120.permissions_dataset.permissions_polygons_new_crs',
                    project_id=credentials.project_id, if_exists='append',
                    credentials=credentials, location='US', )

C:\Users\pkuzi\AppData\Local\Temp\ipykernel_41184\3982166189.py:1: UserWarning: Geometry column does not contain geometry.
  voivodeship_map['geometry'] = voivodeship_map['geometry'].apply(lambda geom: dumps(geom))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [23]:
county_map = gpd.read_file(r'powiaty\powiaty.shp')
county_map = county_map.to_crs(4326)
county_map['geometry'] = county_map['geometry'].apply(lambda geom: dumps(geom))

C:\Users\pkuzi\AppData\Local\Temp\ipykernel_41184\347654315.py:3: UserWarning: Geometry column does not contain geometry.
  county_map['geometry'] = county_map['geometry'].apply(lambda geom: dumps(geom))


In [24]:
pandas_gbq.to_gbq(county_map, 'permissions-417120.permissions_dataset.permissions_polygons_new_crs',
                    project_id=credentials.project_id, if_exists='append',
                    credentials=credentials, location='US', )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
